<a href="https://colab.research.google.com/github/edanielacero/SIS-INT-2023-PRAC-3/blob/main/Proyecto_oficial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

In [ ]:
class State:
    def __init__(self, size=4):
        self.size = size
        self.board = [['.' for _ in range(size)] for _ in range(size)]
        # Initialize chips
        middle = size // 2
        self.board[middle-1][middle-1] = self.board[middle][middle] = 'O'
        self.board[middle][middle-1] = self.board[middle-1][middle] = 'X'

        self.letters_column = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'[:size]


    def show_board(self):
        print('  ' + ' '.join(self.letters_column))

        for i, row in enumerate(self.board):
            print(f'{i+1} ', end='')

            for box in row:
                print(box, end=' ')
            print()
        print()

    def letter_to_number_column(self, letter):
        return ord(letter) - 65  # Changes letter to a number of column (0-7)

    def make_move(self, row, column_letter, player):
      row = row - 1
      column = self.letter_to_number_column(column_letter)
      if self.board[row][column] == ".":
        self.board[row][column] = player
        opponent = 'O' if player == 'X' else 'X'
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, 1), (1, -1), (-1, -1)]

        for dir_f, dir_c in directions:
            i, j = row + dir_f, column + dir_c
            if not self.is_inside_board(i, j) or self.board[i][j] != opponent:
                continue

            i, j = i + dir_f, j + dir_c
            while self.is_inside_board(i, j) and self.board[i][j] == opponent:
                i, j = i + dir_f, j + dir_c

            if self.is_inside_board(i, j) and self.board[i][j] == player:
                while (i != row or j != column):
                    i, j = i - dir_f, j - dir_c
                    self.board[i][j] = player

    def get_valid_movements(self, player):
        valid_movements = []

        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == '.':
                    if self.is_valid_movement(i, j, player):
                        row_number = i + 1
                        column_letter = chr(j + 65)  # changes indice to a letter (A-H)
                        valid_movements.append((row_number, column_letter))

        return valid_movements

    def is_valid_movement(self, row, column, player):
        if self.board[row][column] != '.':
            return False

        opponent = 'O' if player == 'X' else 'X'
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, 1), (1, -1), (-1, -1)]

        for dir_f, dir_c in directions:
            i, j = row + dir_f, column + dir_c
            if self.is_inside_board(i, j) and self.board[i][j] == opponent:
                i, j = i + dir_f, j + dir_c
                while self.is_inside_board(i, j) and self.board[i][j] == opponent:
                    i, j = i + dir_f, j + dir_c

                if self.is_inside_board(i, j) and self.board[i][j] == player:
                    return True

        return False

    def is_inside_board(self, row, column):
        return 0 <= row < self.size and 0 <= column < self.size

    def is_game_over(self):
       return self.full_board()

    def full_board(self):
      cont = sum(1 for row in self.board for box in row if box != ".")
      return cont == self.size * self.size

    def get_winner(self):
        x_counter = sum(row.count('X') for row in self.board)
        o_counter = sum(row.count('O') for row in self.board)

        if x_counter > o_counter:
            return 'X'
        elif o_counter > x_counter:
            return 'O'
        else:
            return 'Draw'

    def utility(self):
      x_counter = sum(row.count('X') for row in self.board)
      o_counter = sum(row.count('O') for row in self.board)
      return x_counter - o_counter

In [ ]:
est = State()
est.show_board()
print(est.get_valid_movements('X'))
est.make_move(3, 'D', 'X')
est.show_board()
print(est.is_game_over())
est.utility()

In [ ]:
est = State(4)
est.show_board()

In [ ]:
import copy

## Min max

In [ ]:
def terminal_test(state):
  return state.is_game_over()

In [ ]:
def result(state, action):
  succesor = copy.deepcopy(state)
  succesor.make_move(action[0][0], action[0][1], action[1])
  #succesor.show_board()
  return succesor

In [ ]:
def utility(state):
  return state.utility()

In [ ]:
def max_value(state):
    if terminal_test(state):
        return utility(state)

    v = float('-inf') # small value
    actions = state.get_valid_movements('X')
    print(actions)
    for action in actions:
        print("Im in MAX actions")
        successor = result(state, (action,'X')) # Returns successor state
        eval_min = min_value(successor) # checks the state (evaluate)
        v = max(v, eval_min)
    return v

def min_value(state):
    if terminal_test(state):
        return utility(state)

    v = float('inf') # big value
    actions = state.get_valid_movements('O')
    print(actions)
    for action in actions:
        print("Im in MIN actions")
        successor = result(state, (action,'O')) # Returns successor state
        eval_max = max_value(successor) # checks the state (evaluate)
        v = min(v, eval_max)
    return v

def min_max(state, player):
    if player == "MAX":
        v = []
        actions = state.get_valid_movements('X')
        print()
        for action in actions:
            print("En mix max superior")
            successor = result(state, (action,'X'))
            v.append((min_value(successor), action))
        return max(v, key=lambda x: x[0])[1]

    elif player == "MIN":
        v = []
        actions = state.get_valid_movements('O')
        for action in actions:
            successor = result(state, (action,'O'))
            v.append((max_value(successor), action))
        return min(v, key=lambda x: x[0])[1]

In [ ]:
initial_state = State(4)
min_max(initial_state, "MAX")

Streaming output truncated to the last 5000 lines.
[(1, 'C')]
Estoy en las acciones de min
Estoy en las acciones de min
[(1, 'C'), (2, 'A')]
Estoy en las acciones de max
[]
Estoy en las acciones de max
[(1, 'C')]
Estoy en las acciones de min
Estoy en las acciones de min
[(1, 'B')]
Estoy en las acciones de max
[(2, 'A'), (3, 'A'), (4, 'A')]
Estoy en las acciones de min
[(3, 'A'), (4, 'A')]
Estoy en las acciones de max
[]
Estoy en las acciones de max
[(3, 'A')]
Estoy en las acciones de min
Estoy en las acciones de min
[(4, 'A')]
Estoy en las acciones de max
[(2, 'A')]
Estoy en las acciones de min
Estoy en las acciones de min
[]
Estoy en las acciones de min
[(3, 'A'), (4, 'A')]
Estoy en las acciones de max
[]
Estoy en las acciones de max
[(1, 'B'), (1, 'C'), (3, 'A')]
Estoy en las acciones de min
[(1, 'C'), (3, 'A')]
Estoy en las acciones de max
[(3, 'A')]
Estoy en las acciones de min
Estoy en las acciones de max
[]
Estoy en las acciones de min
[(1, 'B'), (3, 'A')]
Estoy en las acciones d

(1, 'B')

# Min Max 𝞪-𝜷 pruning

In [ ]:
def max_value(state, alpha, beta):
    if terminal_test(state):
        return utility(state)

    v = float('-inf') # Small Value
    actions = state.get_valid_movements('X')
    #print(actions)
    random.shuffle(actions)
    for action in actions:
        if(alpha>=beta):
          continue
        #print("Im in MAX actions")
        successor = result(state, (action,'X')) # Returns successor state
        eval_min = min_value(successor, alpha, beta) # checks the state (evaluate)
        if eval_min > v:
          v = eval_min
          if v > alpha:
            alpha = v
    return v

def min_value(state, alpha, beta):
    if terminal_test(state):
        return utility(state)

    v = float('inf') # Big Value
    actions = state.get_valid_movements('O')
    #print(actions)
    random.shuffle(actions)
    for action in actions:
        if alpha >= beta:
          continue
        #print("Im in MIN actions")
        successor = result(state, (action,'O')) # Returns successor state
        eval_max = max_value(successor, alpha, beta) # checks the state (evaluate)
        if eval_max < v:
          v = eval_max
          if beta > v:
            beta = v
    return v

def min_max_pruning(state, player):
    alpha = float('-inf') # Small Value
    beta = float('inf') # Big Value
    if player == "MAX":
        v = []
        actions = state.get_valid_movements('X')
        print(actions)
        for action in actions:
            successor = result(state, (action,'X'))
            v.append((min_value(successor, alpha, beta), action))
        return max(v, key=lambda x: x[0])[1]

    elif player == "MIN":
        v = []
        actions = state.get_valid_movements('O')
        for action in actions:
            successor = result(state, (action,'O'))
            v.append((max_value(successor,alpha, beta), action))
        return min(v, key=lambda x: x[0])[1]

In [ ]:
initial_state = State(4)
min_max_pruning(initial_state, "MAX")

[(1, 'B'), (2, 'A'), (3, 'D'), (4, 'C')]


(1, 'B')

In [ ]:
def player_vs_ai(board_size):
    state = State(board_size)
    turn = 'O'  #  Starts playing

    while not state.is_game_over():
        state.show_board()

        if turn == 'X':  # Player's turn
            print("It's your turn.")
            valid_movements = state.get_valid_movements('X')
            print("Valid Movements:")
            for i, (row, column) in enumerate(valid_movements):
                print(f"{i+1}. Row: {row}, Column: {column}")

            selection = int(input("Choose your movement (number): ")) - 1
            row, column = valid_movements[selection]
            state.make_move(row, column, 'X')

        else:  # AI's turn
            print("AI's turn.")
            status_copy = copy.deepcopy(state)
            played_move = min_max_pruning(status_copy, 'MIN')
            state.make_move(played_move[0], played_move[1], 'O')
            print("AI chose: ", played_move)
        turn = 'O' if turn == 'X' else 'X'

    state.show_board()
    winner = state.get_winner()

    if winner == 'Draw':
        print("¡ Its a Draw !")
    else:
        print(f"¡The Winner is {winner}!")

In [ ]:
player_vs_ai(4)

  A B C D
1 . . . . 
2 . O X . 
3 . X O . 
4 . . . . 

Turno de la máquina (IA).
La computadora elijio:  (1, 'C')
  A B C D
1 . . O . 
2 . O O . 
3 . X O . 
4 . . . . 

Es tu turno.
Movimientos válidos:
1. Fila: 1, Columna: B
2. Fila: 1, Columna: D
3. Fila: 3, Columna: D
